<a href="https://colab.research.google.com/github/shawal-mbalire/marconi_internship_notebooks/blob/main/08finetuningWav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

('total',
 10.127777777777698,
 'valid',
 2.0272222222222056,
 'train',
 7.111111111111056,
 'test',
 0.9894444444444359)


In [ ]:
%%capture
!pip install datasets
!pip install transformers
# !pip install huggingface_hub
!pip install torchaudio
!pip install librosa
!pip install jiwer

You grant access to your collab HF_TOKEN

In [ ]:
from datasets import load_dataset, load_metric, Audio

runyankore_train = load_dataset("Shawal777/yogera_runyankore_ailab", "default", split="train+validation")
runyankore_test = load_dataset("Shawal777/yogera_runyankore_ailab", "default", split="test")

Generating train split:   0%|          | 0/1310 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/188 [00:00<?, ? examples/s]

In [ ]:
runyankore_train_clean = runyankore_train.remove_columns(['image_url', 'contributor_id', 'gender', 'age_group', 'language', 'voice_clip', 'duration', 'up_votes', 'down_votes'])
runyankore_test_clean  = runyankore_test.remove_columns(['image_url', 'contributor_id', 'gender', 'age_group', 'language', 'voice_clip', 'duration', 'up_votes', 'down_votes'])

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\\n]'

def remove_special_characters(batch):
    batch["transcript"] = re.sub(chars_to_ignore_regex, '', batch["transcript"]).lower() + " "
    return batch

runyankore_train_clean = runyankore_train_clean.map(remove_special_characters)
runyankore_test_clean  = runyankore_test_clean.map(remove_special_characters)

Map:   0%|          | 0/1684 [00:00<?, ? examples/s]

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcript"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = runyankore_train_clean.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=runyankore_train_clean.column_names
)
vocab_test = runyankore_test_clean.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=runyankore_test_clean.column_names
)
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

Map:   0%|          | 0/1684 [00:00<?, ? examples/s]

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

{' ': 0,
 'r': 1,
 'f': 2,
 'n': 3,
 '3': 4,
 'l': 5,
 'q': 6,
 'j': 7,
 'u': 8,
 'd': 9,
 'o': 10,
 '2': 11,
 'y': 12,
 'g': 13,
 'k': 14,
 'p': 15,
 'i': 16,
 'x': 17,
 'z': 18,
 'w': 19,
 'b': 20,
 "'": 21,
 'h': 22,
 'c': 23,
 'v': 24,
 '4': 25,
 's': 26,
 'm': 27,
 't': 28,
 '0': 29,
 'a': 30,
 'e': 31}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

34

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True
)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer
)

In [ ]:
from datasets import Audio

runyankore_train_clean = runyankore_train_clean.cast_column("link", Audio(sampling_rate=16000))
runyankore_test_clean = runyankore_test_clean.cast_column("link", Audio(sampling_rate=16000))
runyankore_train_clean[0]["link"]

{'path': 'https://storage.googleapis.com/yogera_voices_backup/Runyankole-Rukiga/yogera_image_audio_20230825_101638.084919_1448.wav',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.01519775,
        -0.01211548, -0.02130127]),
 'sampling_rate': 16000}

In [ ]:
runyankore_train_clean[0]["transcript"]

"omukishushani nindeebamu aha maisho y'enjojo harimu nebinyatsi enyuma hariyo n'emiti "

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(runyankore_train_clean)-1)

print(runyankore_train_clean[rand_int]["transcript"])
ipd.Audio(data=runyankore_train_clean[rand_int]["link"]["array"], autoplay=True, rate=16000)

eki n'ekikopo ningira kirimu ekyokunywa eki bamuteraho eki aragumenywisa nanywesereza kiri aha meeza haza eky'okunywisa kiri nkaha aha kapapura 


In [ ]:
def prepare_dataset(batch):
    audio = batch["link"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids
    return batch

In [ ]:
runyankore_train_clean = runyankore_train_clean.map(prepare_dataset)
runyankore_test_clean  = runyankore_test_clean.map(prepare_dataset)

Map:   0%|          | 0/1684 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
# max_input_length_in_sec = 5.0
# common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

# Training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]